In [1]:
import pandas as pd
! pip install wikipedia
! pip install lxml
import lxml
import wikipedia as wp
import requests


!pip install geocoder
import geocoder

!conda install -c conda-forge geopy=1.19.0 --yes 
from geopy.geocoders import Nominatim 

!conda install -c conda-forge --no-deps altair --yes
!conda install -c conda-forge --no-deps vincent --yes

!conda install -c conda-forge --no


-deps folium=0.5.0 --yes
import folium

from sklearn.cluster import KMeans

     |████████████████████████████████| 112kB 32.2MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia
     |████████████████████████████████| 5.8MB 5.4MB/s eta 0:00:01██████████████████████████████▊| 5.7MB 5.4MB/s eta 0:00:01


# Part 1 of Assignment

In [2]:
df = pd.read_html('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050.', header = 0)[0]

In [3]:
df=df[df['Borough']!='Not assigned']
df[df['Neighbourhood']=='Not assigned']['Neighbourhood']=df[df['Neighbourhood']=='Not assigned']['Borough']

In [4]:
df=df.groupby(['Postcode','Borough'])['Neighbourhood'].agg(lambda x:', '.join(list(x))).to_frame()

In [5]:
df=df.reset_index()
df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [6]:
df.shape

(103, 3)

# Part 2 of Assignment 

In [8]:
#for postal_code in list(df['Postcode']):
#    lat_lng_coords=None
#    while(lat_lng_coords is None):
#        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#        lat_lng_coords = g.latlng
#    df[df['Postcode']==postal_code]['latitude'] = lat_lng_coords[0]
#    df[df['Postcode']==postal_code]['longitude'] = lat_lng_coords[1]

### Since Geocoder package is unreliable and slow we would use CSV file of Geospatial data of Toronto

In [9]:
!wget -q -O 'toronto_geospatial_data.csv' http://cocl.us/Geospatial_data

In [10]:
df1=pd.read_csv('toronto_geospatial_data.csv')
df1=df1.rename(columns={'Postal Code':'Postcode'})

In [11]:
df_total=pd.merge(df,df1,left_on='Postcode',right_on='Postcode')

In [12]:
df_total

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


# Part 3 of Assignment

### We only use those Boroughs that have Toronto in their name 

In [25]:
df_total=df_total[df_total['Borough'].str.endswith('Toronto')].reset_index(drop=True)
df_total.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### The Next cell has the Foursquare Credentials

In [122]:
# The code was removed by Watson Studio for sharing.

In [63]:
LIMIT = 30 
radius = 500
VERSION='20200327'

lst=[]
for name,lat,long in zip(df_total['Neighbourhood'], df_total['Latitude'], df_total['Longitude']):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    long, 
    radius, 
    LIMIT)
    results = requests.get(url).json()['response']['groups'][0]['items']
    for i in results:
        lst.append((name,lat,long,i['venue']['name'],i['venue']['location']['lat'],i['venue']['location']['lng'],i['venue']['categories'][0]['name']))

In [65]:
df_toronto=pd.DataFrame(lst)
df_toronto.columns=['Neighbourhood','Neighbourhood Latitude','Neighbourhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude','Venue Category']

In [67]:
df_toronto.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Park,43.675278,-79.294647,Park
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


In [70]:
df_tornoto_onehot=pd.get_dummies(df_toronto['Venue Category'])
df_tornoto_onehot['Neighbourhood']=df_toronto['Neighbourhood']
df_tornoto_onehot=df_tornoto_onehot[['Neighbourhood']+list(df_tornoto_onehot.columns)[:-1]]

In [81]:
df_toronto_onehot=df_tornoto_onehot.groupby('Neighbourhood').agg('mean').reset_index()

In [85]:
df_final=pd.merge(df_total,df_toronto_onehot,left_on='Neighbourhood',right_on='Neighbourhood')
df_final.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0.0,0.0,0.0,0.0,0.0,...,0.0,0.200000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0.0,0.0,0.0,0.0,0.0,...,0.0,0.033333,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.033333
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.033333
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000


In [109]:
df_clustering=df_final.drop(['Neighbourhood','Postcode','Borough','Latitude','Longitude'],axis=1)
kmeans = KMeans(init="k-means++", n_clusters=4, n_init=12, random_state=0).fit(df_clustering)

In [123]:
address = 'Toronto, Ontario Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.19.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


## Visualizing the clusters

In [120]:
map_toronto_neighbourhood = folium.Map(location=[latitude, longitude], zoom_start=12)

colors=['blue','green','red','yellow']

for lat, long, poi, cluster in zip(df_final['Latitude'], df_final['Longitude'], df_final['Neighbourhood'], kmeans.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color=colors[cluster],
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(map_toronto_neighbourhood)
       
map_toronto_neighbourhood